# Financial Transaction Workflow 🏦

## Libraries

In [4]:
import json
import numpy as np
import random
import pandas as pd
import mysql.connector
from neo4j import GraphDatabase

## MySQL Database Connection

In [5]:
json_file_path = r'C:\Users\Catello\Desktop\Protfolio\Project_2\database_config_mysql.json'

with open(json_file_path, 'r') as json_file:
    database_config = json.load(json_file)

mydb = mysql.connector.connect(**database_config)
mycursor = mydb.cursor()

## SQL Query

In [6]:
mycursor.execute("USE project_2")

In [7]:
# Query to show tables
show_tables_query = "SHOW TABLES"

# Execute the query
mycursor.execute(show_tables_query)

# Fetch all the rows
tables = mycursor.fetchall()

# Create a Pandas DataFrame
df_tables = pd.DataFrame(tables, columns=["Tables"])

# Display the DataFrame
print(df_tables)

                   Tables
0  financial_transactions


In [8]:
# Execute the query
query = '''
SELECT *
FROM financial_transactions
'''

# Load results into a Pandas DataFrame
transaction_DataFrame = pd.read_sql_query(query, mydb)

# Display the DataFrame
transaction_DataFrame.head()

C:\Users\Catello\AppData\Local\Temp\ipykernel_10052\1671986894.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  transaction_DataFrame = pd.read_sql_query(query, mydb)


,transaction_id,merchant_number,transaction_amount,transaction_category
0,1,369,625510,bank_deposit
1,2,1870,681505,international_money_transfer
2,3,1789,897404,domestic_money_transfer
3,4,972,1141892,bank_deposit
4,5,2379,749053,international_money_transfer


## Neo4j Database Connection

In [12]:
# Load database configuration from JSON file
json_file_path = r'C:\Users\Catello\Desktop\Protfolio\Project_2\database_config_neo4j.json'

with open(json_file_path, 'r') as json_file:
    database_config = json.load(json_file)

# Extracting authentication details from the dictionary
auth_details = database_config.get("auth", {})
uri = database_config.get("uri", "")

# Connect to Neo4j
data_base_connection = GraphDatabase.driver(uri, auth=(auth_details.get("user", ""), auth_details.get("password", "")))


## Neo4j Query

In [11]:
transaction_list = transaction_DataFrame.values.tolist()

transaction_execution_commands = []

for i in transaction_list:
    neo4j_create_statemenet = "create (t:Transaction {transaction_id:" + str(i[0]) +", vendor_number:  " + str(i[1]) +", transaction_amount: " + str(i[2]) +", transaction_type: '" + str(i[3]) + "'})"
    transaction_execution_commands.append(neo4j_create_statemenet)

    
def execute_transactions(transaction_execution_commands):
    session = data_base_connection.session()    
    for i in transaction_execution_commands:
        session.run(i)

        
execute_transactions(transaction_execution_commands)
 

In [12]:
unique_transaction_types = ["match (t:Transaction) return distinct t.transaction_type"]

def execute_transactions(transaction_execution_commands, return_result = False):
    session = data_base_connection.session()
    return_list = []
    
    for i in transaction_execution_commands:
        transaction_result = session.run(i)
        return_list = [j[0] for j in transaction_result]
    
    if return_result:
        return return_list

        
unique_transaction_results = execute_transactions(unique_transaction_types, True)
print(unique_transaction_results)

['bank_deposit', 'international_money_transfer', 'domestic_money_transfer', 'ATM_withdrawal']


In [13]:
create_labels_commands = []

for i in unique_transaction_results:
    create_labels_commands.append("match (t:Transaction) where t.transaction_type = '" + i + "' set t :" + i )
    
print(create_labels_commands)


execute_transactions(create_labels_commands)

["match (t:Transaction) where t.transaction_type = 'bank_deposit' set t :bank_deposit", "match (t:Transaction) where t.transaction_type = 'international_money_transfer' set t :international_money_transfer", "match (t:Transaction) where t.transaction_type = 'domestic_money_transfer' set t :domestic_money_transfer", "match (t:Transaction) where t.transaction_type = 'ATM_withdrawal' set t :ATM_withdrawal"]


In [13]:
# Unique vendor ID command
unique_vendor_id_command = ["MATCH (t:Transaction) RETURN DISTINCT t.vendor_number"]

# Execute the command to get unique vendor IDs
list_unique_vendor_id = execute_transactions(unique_vendor_id_command, True)

# Create process mapping
create_process_mapping = []

for i in list_unique_vendor_id:
    create_process_mapping.append(
        "MATCH (t:Transaction) WHERE t.vendor_number = '" + str(i) + "' "
        "WITH t ORDER BY t.id "
        "WITH COLLECT(t) AS ordered_transactions "
        "FOREACH (ignore IN RANGE(0, SIZE(ordered_transactions) - 2) | "
        "    FOREACH (j IN [ordered_transactions[ignore]] | "
        "        FOREACH (k IN [ordered_transactions[ignore + 1]] | "
        "            MERGE (j)-[:PRECESS_WORKFLOW]->(k)"
        "        )"
        "    )"
        ")"
    )
    
def execute_transactions(transaction_execution_commands):
    session = data_base_connection.session()    
    for i in transaction_execution_commands:
        session.run(i)
    
    
# Execute process mapping
execute_transactions(create_process_mapping)


NameError: name 'execute_transactions' is not defined